# Serialization 

### Topics

- serializing objects
- deserializing objects
- pickle
- JSON

## Serializing objects

- objects created in memory are not persistent
    - they hold a lot of important information/states of objects as the program is being executed
- to make an object persistent, we need to create a series of bytes that represent the state of the object, and write those bytes to a file
    - need to encode objects (**serializing**)
    - also need to decode objects from series of bytes (**deserializing**)
- web services often use a service described as **RESTful**
    - REST - REpresentational State Transfer
- the server and client will exchange representation of objects using RESTful services
- several ways to serialize and objects
- Pickle, JSON, XML, etc.

## Serializing objects using pickle

- Python `pickle` module is an object-oriented way to store object state directly in a special storage format
- essentially converts an object's state (and all the state of all the objects it holds as attributes) into a series of bytes
    - these bytes then can be transported or stored however we see fit
- pickle file are for temporary persistence
    - Pickled file using Python 3.7 may not be unpickled by newer version of Python or vice versa
- NOTE - pickle module is not secure
    - only unpickle data you trust!

In [15]:
import pickle

In [16]:
help(pickle)

Help on module pickle:

NAME
    pickle - Create portable serialized representations of Python objects.

MODULE REFERENCE
    https://docs.python.org/3.10/library/pickle.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    See module copyreg for a mechanism for registering custom picklers.
    See module pickletools source for extensive comments.
    
    Classes:
    
        Pickler
        Unpickler
    
    Functions:
    
        dump(object, file)
        dumps(object) -> string
        load(file) -> object
        loads(bytes) -> object
    
    Misc variables:
    
        __version__
        format_version
        compatible_formats

CLASSES
    builtins.Exception(builtins.BaseException)


In [3]:
some_data = ['a list', 'containing', 5, 'items', {"including": ("str", "int", "dict", "list", "tuple")}]

In [4]:
some_data

['a list',
 'containing',
 5,
 'items',
 {'including': ('str', 'int', 'dict', 'list', 'tuple')}]

In [7]:
# let's pickle/serialize the object list object
with open("pickled_list.pkl", 'wb') as file:
    pickle.dump(some_data, file)

In [8]:
# lets deserialize/load the object from file
with open("pickled_list.pkl", 'rb') as file:
    loaded_list = pickle.load(file)

In [9]:
type(loaded_list)

list

In [10]:
loaded_list

['a list',
 'containing',
 5,
 'items',
 {'including': ('str', 'int', 'dict', 'list', 'tuple')}]

In [12]:
some_data == loaded_list

True

In [13]:
loaded_list.append([1, 2, 3])

In [14]:
loaded_list

['a list',
 'containing',
 5,
 'items',
 {'including': ('str', 'int', 'dict', 'list', 'tuple')},
 [1, 2, 3]]

## Serializing objects using JSON

- there are many text-based format to exchange data
- XML - Extensive Markup Language (XML)
- YAML - Yet Another Markup Language
- CSV - Comma-Separated Value
- most of these techniques have obscure features that can be exploited from security point of view
    - allow arbitrary commands to be executed on the host machine

### JSON - JavaScript Object Notation

- human-readable text-based format for exchanging data
- one of the most popular tecnique used by RESTful API services
- JSON as the name says is more popular in JavaScript language to transfer data to a browswer from the web server
- JSON though popular is not as robust as the **pickle** module
- it can serialize only basic data: 
    - integers, floats, strings and simple containers such as list and dictionaries
- generally, `json` module's functions try to serialize the object's state using the value of the object's `__dict__` attribute
- json provides **dump** and **dumps**, **load** and **loads** functions

In [17]:
import json

In [18]:
help(json)

Help on package json:

NAME
    json

MODULE REFERENCE
    https://docs.python.org/3.10/library/json.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    JSON (JavaScript Object Notation) <https://json.org> is a subset of
    JavaScript syntax (ECMA-262 3rd edition) used as a lightweight data
    interchange format.
    
    :mod:`json` exposes an API familiar to users of the standard library
    :mod:`marshal` and :mod:`pickle` modules.  It is derived from a
    version of the externally maintained simplejson library.
    
    Encoding basic Python object hierarchies::
    
        >>> import json
        >>> json.dumps(['foo', {'bar': ('baz', None, 1.0, 2)}])
        '["foo", {"bar": ["baz", nul

In [19]:
class Contact:
    def __init__(self, first, last):
        self.first = first
        self.last = last
        
    @property
    def full_name(self):
        return f'{self.first} {self.last}'

In [20]:
c = Contact("John", "Smith")

In [22]:
c.full_name

'John Smith'

In [23]:
json.dumps(c.__dict__)

'{"first": "John", "last": "Smith"}'

In [25]:
# better approach
from typing import Any
import json

class ContactEncoder(json.JSONEncoder):
    def default(self, obj: Any) -> Any:
        if isinstance(obj, Contact):
            return {
                "__class__": "Contact",
                "first": obj.first,
                "last": obj.last,
                "full_name": obj.full_name,
            }
        return super().default(obj)

In [26]:
c = Contact("John", "Smith")
text = json.dumps(c, cls=ContactEncoder)

In [27]:
text

'{"__class__": "Contact", "first": "John", "last": "Smith", "full_name": "John Smith"}'

In [28]:
def decode_contact(json_object: Any) -> Any:
    if json_object.get("__class__") == "Contact":
        return Contact(json_object["first"], json_object["last"])
    else:
        return json_object

In [29]:
c2 = json.loads(text, object_hook=decode_contact)

In [30]:
c2.full_name

'John Smith'

In [31]:
some_text = ('{"__class__": "Contact", "first": "Milli", "last": "Dale", '
            '"full_name": "Milli Dale"}')

In [32]:
c3 = json.loads(some_text, object_hook=decode_contact)

In [33]:
c3.full_name

'Milli Dale'

In [34]:
help(c3)

Help on Contact in module __main__ object:

class Contact(builtins.object)
 |  Contact(first, last)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, first, last)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |  
 |  full_name
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



## Weather App

- Weather data sources: https://weatherstack.com/, https://openweathermap.org/, gogole.com, etc.
- openweathermap API takes couple of hours to activate
- Most provide free API access with some limitations
- replace access_key/api_id with your API key

In [71]:
import json

In [9]:
# get current weather info for Grand Junction, Colorado
# using weatherstack.com API

import requests

params = {
  'access_key': '', # FIXME
  'query': 'Grand Junction, Colorado'
}

api_result = requests.get('http://api.weatherstack.com/current', params) 
# only allows http access for with free account

api_response = api_result.json()

In [49]:
api_response

{'coord': {'lon': -108.5507, 'lat': 39.0639},
 'weather': [{'id': 600,
   'main': 'Snow',
   'description': 'light snow',
   'icon': '13n'}],
 'base': 'stations',
 'main': {'temp': 273.81,
  'feels_like': 269.66,
  'temp_min': 272.15,
  'temp_max': 275.18,
  'pressure': 1009,
  'humidity': 67},
 'visibility': 10000,
 'wind': {'speed': 4.02, 'deg': 293, 'gust': 6.26},
 'snow': {'1h': 0.11},
 'clouds': {'all': 75},
 'dt': 1680662257,
 'sys': {'type': 2,
  'id': 2020637,
  'country': 'US',
  'sunrise': 1680612849,
  'sunset': 1680658794},
 'timezone': -21600,
 'id': 5423573,
 'name': 'Grand Junction',
 'cod': 200}

In [48]:
# print better
print(json.dumps(api_response, indent=2))

{
  "coord": {
    "lon": -108.5507,
    "lat": 39.0639
  },
  "weather": [
    {
      "id": 600,
      "main": "Snow",
      "description": "light snow",
      "icon": "13n"
    }
  ],
  "base": "stations",
  "main": {
    "temp": 273.81,
    "feels_like": 269.66,
    "temp_min": 272.15,
    "temp_max": 275.18,
    "pressure": 1009,
    "humidity": 67
  },
  "visibility": 10000,
  "wind": {
    "speed": 4.02,
    "deg": 293,
    "gust": 6.26
  },
  "snow": {
    "1h": 0.11
  },
  "clouds": {
    "all": 75
  },
  "dt": 1680662257,
  "sys": {
    "type": 2,
    "id": 2020637,
    "country": "US",
    "sunrise": 1680612849,
    "sunset": 1680658794
  },
  "timezone": -21600,
  "id": 5423573,
  "name": "Grand Junction",
  "cod": 200
}


In [14]:
def celciusToFahrenheit(celcius: int) -> float:
    return celcius*(9/5)+32 

In [11]:
celciusTemp = api_response['current']['temperature']

In [12]:
print('Current temperature in %s is %d℃' % (api_response['location']['name'], celciusTemp))

Current temperature in Grand Junction is 3℃


In [15]:
print(f"Current temperature in {api_response['location']['name']} is \
    {celciusToFahrenheit(celciusTemp)} {chr(8457)}")

Current temperature in Grand Junction is 37.4 ℉


In [29]:
# get current weather info for Grand Junction, Colorado
# using openweathermap.org API
import requests

params = {
  'appid': '', # FIXME
  'q': 'Grand Junction, Colorado'
}

url = 'https://api.openweathermap.org/data/2.5/weather'

# must use http for free tier not https
api_result = requests.get(url, params)

api_response = api_result.json()

In [50]:
api_response

{'coord': {'lon': -108.5507, 'lat': 39.0639},
 'weather': [{'id': 600,
   'main': 'Snow',
   'description': 'light snow',
   'icon': '13n'}],
 'base': 'stations',
 'main': {'temp': 273.81,
  'feels_like': 269.66,
  'temp_min': 272.15,
  'temp_max': 275.18,
  'pressure': 1009,
  'humidity': 67},
 'visibility': 10000,
 'wind': {'speed': 4.02, 'deg': 293, 'gust': 6.26},
 'snow': {'1h': 0.11},
 'clouds': {'all': 75},
 'dt': 1680662257,
 'sys': {'type': 2,
  'id': 2020637,
  'country': 'US',
  'sunrise': 1680612849,
  'sunset': 1680658794},
 'timezone': -21600,
 'id': 5423573,
 'name': 'Grand Junction',
 'cod': 200}

In [35]:
import json
print(json.dumps(api_response, indent=2))

{
  "coord": {
    "lon": -108.5507,
    "lat": 39.0639
  },
  "weather": [
    {
      "id": 600,
      "main": "Snow",
      "description": "light snow",
      "icon": "13n"
    }
  ],
  "base": "stations",
  "main": {
    "temp": 273.81,
    "feels_like": 269.66,
    "temp_min": 272.15,
    "temp_max": 275.18,
    "pressure": 1009,
    "humidity": 67
  },
  "visibility": 10000,
  "wind": {
    "speed": 4.02,
    "deg": 293,
    "gust": 6.26
  },
  "snow": {
    "1h": 0.11
  },
  "clouds": {
    "all": 75
  },
  "dt": 1680662257,
  "sys": {
    "type": 2,
    "id": 2020637,
    "country": "US",
    "sunrise": 1680612849,
    "sunset": 1680658794
  },
  "timezone": -21600,
  "id": 5423573,
  "name": "Grand Junction",
  "cod": 200
}


In [39]:
max_temp = api_response['main']['temp_max'] # temperature is in Kelvin

In [40]:
max_temp 

275.18

In [42]:
curr_temp = api_response['main']['temp']

In [43]:
curr_temp

273.81

In [52]:
print(f'Current Temperature in {api_response["name"]} is: {celciusToFahrenheit(curr_temp-273.15):.1f}{chr(8457)}')

Current Temperature in Grand Junction is: 33.2℉


In [38]:
api_response['main']['temp_min']

272.15

## Using Google Search

- When you search for places in Google, it provides some weather info as well

In [81]:
from bs4 import BeautifulSoup
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def getGoogleWeather(city):
    city = city.replace(" ", "+")
    url = f'https://www.google.com/search?q={city}'
    res = requests.get(url, headers=headers)
    print("Searching...\n")
    soup = BeautifulSoup(res.text, 'html.parser')
    location = soup.select('.BBwThe')[0].getText().strip() # search by class
    time = soup.select('#wob_dts')[0].getText().strip() #search by id
    forecast = soup.select('#wob_dc')[0].getText().strip()
    temp = soup.select('#wob_tm')[0].getText().strip()
    return location, time, forecast, temp


In [82]:
city = input("Enter the Name of a City ->  ")
city = city+" weather"
result = getGoogleWeather(city)

Enter the Name of a City ->  Grand Junction
Searching...



In [83]:
result

('Grand Junction, CO', 'Tuesday 9:00\u202fPM', 'Partly cloudy', '31')

### Weather forecast
- openweathermap.org provieds 5-day 3-hour forecast data for freemium account

In [58]:
# get current weather info for Grand Junction, Colorado
# using openweathermap.org API
import requests

def forecastWeather(city) -> json:
    params = {
      'appid': '', # FIXME
      'q': city,
      'units': 'imperial' # uses US metrics
    }

    #url = httpsapi.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&appid={API key}
    url = 'https://api.openweathermap.org/data/2.5/forecast'

    api_result = requests.get(url, params)

    api_response = api_result.json()
    return api_response

In [66]:
city = input('Enter name of a City: ')

Enter name of a City: Grand Junction


In [ ]:
json_response = forecastWeather(city)

In [60]:
json_response

{'cod': '200',
 'message': 0,
 'cnt': 40,
 'list': [{'dt': 1680674400,
   'main': {'temp': 32.05,
    'feels_like': 26.04,
    'temp_min': 30.92,
    'temp_max': 32.05,
    'pressure': 1012,
    'sea_level': 1012,
    'grnd_level': 855,
    'humidity': 59,
    'temp_kf': 0.63},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'clouds': {'all': 44},
   'wind': {'speed': 6.33, 'deg': 310, 'gust': 11.43},
   'visibility': 10000,
   'pop': 0.68,
   'sys': {'pod': 'n'},
   'dt_txt': '2023-04-05 06:00:00'},
  {'dt': 1680685200,
   'main': {'temp': 31.03,
    'feels_like': 31.03,
    'temp_min': 30.24,
    'temp_max': 31.03,
    'pressure': 1016,
    'sea_level': 1016,
    'grnd_level': 857,
    'humidity': 61,
    'temp_kf': 0.44},
   'weather': [{'id': 803,
     'main': 'Clouds',
     'description': 'broken clouds',
     'icon': '04n'}],
   'clouds': {'all': 73},
   'wind': {'speed': 1.19, 'deg': 305, 'gust': 3.76},
   'visib

In [70]:
from datetime import datetime

print('Forecast Result for: ', json_response['city']['name'])
for forecast in json_response['list']:
    print('Day: ', datetime.fromtimestamp(forecast['dt']))
    feelsLike = f"{forecast['main']['feels_like']} {chr(8457)}"
    temp = f"{forecast['main']['temp']} {chr(8457)}"
    print(f"Feels like: {feelsLike} Temperature: {temp}")

Forecast Result for:  Grand Junction
Day:  2023-04-05 00:00:00
Feels like: 26.04 ℉ Temperature: 32.05 ℉
Day:  2023-04-05 03:00:00
Feels like: 31.03 ℉ Temperature: 31.03 ℉
Day:  2023-04-05 06:00:00
Feels like: 29.8 ℉ Temperature: 29.8 ℉
Day:  2023-04-05 09:00:00
Feels like: 29.12 ℉ Temperature: 32.14 ℉
Day:  2023-04-05 12:00:00
Feels like: 32.99 ℉ Temperature: 36.72 ℉
Day:  2023-04-05 15:00:00
Feels like: 35.26 ℉ Temperature: 40.53 ℉
Day:  2023-04-05 18:00:00
Feels like: 32.99 ℉ Temperature: 38.66 ℉
Day:  2023-04-05 21:00:00
Feels like: 35.15 ℉ Temperature: 35.15 ℉
Day:  2023-04-06 00:00:00
Feels like: 29.26 ℉ Temperature: 33.01 ℉
Day:  2023-04-06 03:00:00
Feels like: 29.61 ℉ Temperature: 32.41 ℉
Day:  2023-04-06 06:00:00
Feels like: 26.91 ℉ Temperature: 31.3 ℉
Day:  2023-04-06 09:00:00
Feels like: 30.07 ℉ Temperature: 36.12 ℉
Day:  2023-04-06 12:00:00
Feels like: 40.33 ℉ Temperature: 43.79 ℉
Day:  2023-04-06 15:00:00
Feels like: 47.8 ℉ Temperature: 49.21 ℉
Day:  2023-04-06 18:00:00
Fee

## Exercise

1. Find average temperate for Grand Junction in the 1st week of January 2000.
2. Create a simple weather app using any API of your choice.